In [ ]:
from h5py import File as h5File

from numpy import argsort, linspace
from scipy.interpolate import interp1d

from matplotlib.pyplot import close, subplots

from os.path import expanduser, join

import time

close('all')

select_tags = ['VDH', 'IDH', 'PINP', 'PTUB', 'PCAS', 'YRUN']
tag_idx = {'VDH': 0, 'IDH': 1, 'PINP': 2, 'PTUB': 3, 'PCAS': 4, 'YRUN': 5}

file_folder = join('Downloads', 'Hackathon2017', 'data')
file_path = join(expanduser('~'), file_folder)

def interpolate(time_arr, data_arr):
    function = interp1d(time_arr, data_arr)

    time_limits = [time_arr.min(), time_arr.max()]
    time_numsamp = time_limits[1] - time_limits[0] + 1

    time_arr_new = linspace(time_limits[0], time_limits[1], time_numsamp)

    data_arr_new = function(time_arr_new)

    return time_arr_new, data_arr_new

with h5File(join(file_path, 'esp_data.h5'), 'r') as fp:
    field_arr = list(fp.keys())

    for field in field_arr:
        esp_arr = list(fp[field].keys())

        for esp in esp_arr:
            print('Doing ESP: ', esp)
            fig, ax = subplots(6, 1, figsize=(16, 14), facecolor='white', sharex=True)
            sensor_arr = list(fp[field][esp].keys())
            sensor_arr = select_tags

            for sensor in sensor_arr:
                try:
                    label_curr = sensor
                    time_arr = fp[field][esp][sensor][:, 0]
                    data_arr = fp[field][esp][sensor][:, 1]

                    sorted_idx = argsort(time_arr)

                    time_arr = time_arr[sorted_idx]
                    data_arr = data_arr[sorted_idx]

                    if time_arr.size > 1:
                        time_interp, data_interp = interpolate(time_arr, data_arr)
                    else:
                        time_interp = time_arr * 1
                        data_interp = data_arr * 1

                    if label_curr is 'VDH':
                        ax[tag_idx[label_curr]].set_ylim([0, 2])

                    ax[tag_idx[label_curr]].plot(time_interp, data_interp)
                    ax[tag_idx[label_curr]].set_ylabel(label_curr)

                except KeyError:
                    print("no key")

            fig.tight_layout()
            fig.savefig('images/esp' + esp + '.png')
            close(fig)
